In [11]:
# PYTHON PACKAGES
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
import google.auth.transport.requests 
from google.auth.transport.requests import AuthorizedSession
import gspread
from datetime import datetime


In [12]:
# INITIAL CONFIGS
now = datetime.now()
today = now.strftime("%d/%m/%Y") #GET THE CURRENT DAY
path_sheets = "H:\My Drive\MBA\DAS 0013\sheets-credentials.json" # LOCATION OF .JSON FILE WITH CREDENTIALS OF GOOGLE API
path_spotify = "H:\My Drive\MBA\DAS 0013\spotify_confidential.txt.txt" # CREATE A .TXT FILE WITH THE CLIENT ID AND SECRETE OF SPOTIFY API
cid = open(path_spotify, "r").read().split('\n')[0] #SPOTIFY CREDENTIALS
secret=open(path_spotify, "r").read().split('\n')[1] #SPOTIFY CREDENTIALS
key = '18MFV_mk8PAem4aaO0Psl6SMTmH6TCsdLNZGJvLsGNVQ' #GOOGLE SHEETS KEY


In [13]:
def pandas_df_to_googlesheets_(key,Data,path=path_sheets):
    Dataset = Data.copy().astype(str) # COPY DATAFRAME
    Valores = Dataset.values #GET ALL VALUES
    Columnas = Dataset.columns.values # GET ALL COLUMNS NAMES

    scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']; #Apis autorizadas
    credentials = Credentials.from_service_account_file(path,scopes=scopes); #Se lee lacredencial
    
    request = google.auth.transport.requests.Request()
   
    credentials.refresh(request)

    gc = gspread.authorize(credentials)
    sht = gc.open_by_key(str(key)); # GET THE SHEETS WITH THE KEY

    worksheet = sht.worksheet('RawData'); #EXTRACT THE SPECIFIED SHEETS

    response = worksheet.get_values('A1:A') #GET ALL VALUES IN FIRST COLUMN

    last_row = str(len(response)+1) # GET THE LAST EMPTY ROW AVAILABLE

    worksheet.update("A" + last_row,Valores.tolist()) #UPDATE SHEET WITH ONLY THE VALUES STARTING IN THE FIRST EMPTY ROW

In [14]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [15]:
#GET TOP 50 TRACKS PLAYLIST
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [16]:
#CREATE EMPTY LIST
lst_data = []

In [8]:
# LOOP THROUGH TRACKS AND CREATING DATASET WITH PROCESSED DATA
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    artist_url = artist_info["external_urls"]['spotify']
    artist_followers = artist_info["followers"]['total']
    artist_image = artist_info["images"][0]['url']



    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    """
    data = [
        ['track_uri', track_uri],
        ['track_name', track_name],
        ['artist_uri', artist_uri],
        ['artist_info', artist_info],
        ['artist_name', artist_name],
        ['artist_pop', artist_pop],
        ['artist_genres', artist_genres],
        ['album', album],
        ['track_pop', track_pop]
        ]
    """
    #COMPILE ALL VARIABLES IN A LIST
    data = [
        today,
        track_uri,
        track_name,
        artist_uri,
        artist_info,
        artist_name,
        artist_pop,
        artist_genres,
        album,
        track_pop,
        artist_url,
        artist_followers,
        artist_image
        ]
    # CREATE A PANDAS DATAFRAME OF EACH LIST 
    df1 = pd.DataFrame([data],columns=['date','track_uri','track_name','artist_uri','artist_info','artist_name','artist_pop','artist_genres','album','track_pop','artist_url','artist_followers','artist_image'])
    lst_data.append(df1) # COMPILE EACH DATAFRAME IN THE EMPTY LIST PREVIOUSLY CREATED

df = pd.concat(lst_data, ignore_index=True) #JOIN ALL DATAFRAMES IN THE LIST IN A SINGLE FINAL ONE
df.reset_index(inplace=True)

In [9]:
#SEE FINAL DATASET
df.head()

,index,date,track_uri,track_name,artist_uri,artist_info,artist_name,artist_pop,artist_genres,album,track_pop,artist_url,artist_followers,artist_image
0,0,16/10/2022,spotify:track:3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),spotify:artist:2wY79sveU1sp5g7SokKOiI,{'external_urls': {'spotify': 'https://open.sp...,Sam Smith,88,"[dance pop, pop, uk pop]",Unholy (feat. Kim Petras),100,https://open.spotify.com/artist/2wY79sveU1sp5g...,20396490,https://i.scdn.co/image/ab6761610000e5eb53e9f1...
1,1,16/10/2022,spotify:track:4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),spotify:artist:1Cs0zKBU1kc0i8ypK3B9ai,{'external_urls': {'spotify': 'https://open.sp...,David Guetta,89,"[big room, dance pop, edm, pop, pop dance]",I'm Good (Blue),99,https://open.spotify.com/artist/1Cs0zKBU1kc0i8...,24827466,https://i.scdn.co/image/ab6761610000e5eb75348e...
2,2,16/10/2022,spotify:track:4Dvkj6JhhA12EX05fT7y2e,As It Was,spotify:artist:6KImCVD70vtIoJWnq6nGn3,{'external_urls': {'spotify': 'https://open.sp...,Harry Styles,91,[pop],Harry's House,93,https://open.spotify.com/artist/6KImCVD70vtIoJ...,23772442,https://i.scdn.co/image/ab6761610000e5ebf7db7c...
3,3,16/10/2022,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52",spotify:artist:716NhGYqD1jl2wI1Qkgq36,{'external_urls': {'spotify': 'https://open.sp...,Bizarrap,86,"[argentine hip hop, pop venezolano, trap argen...","Quevedo: Bzrp Music Sessions, Vol. 52",99,https://open.spotify.com/artist/716NhGYqD1jl2w...,7338028,https://i.scdn.co/image/ab6761610000e5eb12085f...
4,4,16/10/2022,spotify:track:5ww2BF9slyYgNOk37BlC4u,La Bachata,spotify:artist:0tmwSHipWxN12fsoLcFU3B,{'external_urls': {'spotify': 'https://open.sp...,Manuel Turizo,85,"[colombian pop, reggaeton, reggaeton colombian...",La Bachata,98,https://open.spotify.com/artist/0tmwSHipWxN12f...,10493876,https://i.scdn.co/image/ab6761610000e5ebd9a4d3...


In [10]:
#PUSH DATA SET TO GOOGLE SPREAD SHEETS
pandas_df_to_googlesheets_(key,df)